In [1]:
import argparse

hparams = argparse.Namespace()
hparams.eval_beams: int  = 4
hparams.learning_rate: float = 2e-5
hparams.adam_epsilon: float = 1e-8
hparams.warmup_steps: int = 0
hparams.weight_decay: float = 0.0
hparams.max_epochs: int = 1
hparams.gpus: int = 1
hparams.train_batch_size: int = 16
hparams.eval_batch_size: int = 16
hparams.accumulate_grad_batches: int = 1
hparams.early_stopping: bool =True
    
hparams.max_seq_length: int = 64
hparams.LongTermMemory_size :int = 1000
hparams.age_noise:float = 4.0
hparams.reward_scaling_factor:float = 10.0
hparams.reward_age_base:float = 10.0
hparams.age_discount:float = 1.0
hparams.hit_th:float = 0.9
hparams.max_seq_length : int = 64
hparams.max_dialogue_length: int = 3
    
hparams.gamma : float= 0.98
    
hparams.EMBEDDING_DIM:int = 100
hparams.N_FILTERS:int = 100
hparams.FILTER_SIZES = [3, 5, 7]
hparams.MIN_LENGTH:int = 7
hparams.OUTPUT_DIM:int = 1
hparams.DROPOUT:float = 0.5

In [2]:
from chatbot import Chatbot
chatbot = Chatbot(hparams)

RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at /tmp/pip-req-build-ufslq_a9/caffe2/serialize/inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at /tmp/pip-req-build-ufslq_a9/caffe2/serialize/inline_container.cc:132)
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) + 0x6d (0x7f262037d2ad in /home/insublee/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: caffe2::serialize::PyTorchStreamReader::init() + 0x25db (0x7f25f018aa1b in /home/insublee/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #2: caffe2::serialize::PyTorchStreamReader::PyTorchStreamReader(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) + 0x7b (0x7

In [ ]:
from chatbot.data import DataModule
dm = DataModule(hparams)
dm.prepare_data()
dm.setup('fit')

In [ ]:
# data check
batch = next(iter(dm.train_dataloader()))
for i in batch.keys():
    print(i, batch[i].size())

In [ ]:
# forward test
forward_out = chatbot(**batch)
print(forward_out.keys()) # odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [ ]:
# training step test
loss = chatbot.training_step(batch,0)
print(loss) # tensor(14.2616, grad_fn=<AddBackward0>)

In [ ]:
# validation step test
loss = chatbot.validation_step(batch,0)
print(loss) # tensor(14.2616, grad_fn=<AddBackward0>)

In [ ]:
# generation test1. no input
outputs = chatbot.model.generate(max_length=40)
print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# generation test2. yes input
input_text = "hello my name is insub, and"
tokenized_text = dm.tokenizer([input_text], return_tensors="pt").input_ids
outputs = chatbot.model.generate(input_ids=tokenized_text, max_length=40)
print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import pytorch_lightning as pl
# trainer loading and fit
#trainer = pl.Trainer(gpus=-1, auto_select_gpus=True)
#trainer = pl.Trainer(gpus=4, accelerator='ddp2')
#trainer = pl.Trainer(gpus=1, max_epochs=2)
trainer = pl.Trainer(gpus=1, max_epochs=1)
#trainer.fit(chatbot, dm.train_dataloader(), dm.val_dataloader())
trainer.fit(chatbot, dm)

In [ ]:
# generation test3. after training no input
outputs = chatbot.model.generate(max_length=40)
print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# generation test4. after training yes input
input_text = "hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape ."
tokenized_text = dm.tokenizer([input_text], return_tensors="pt").input_ids
outputs = chatbot.model.generate(input_ids=tokenized_text, max_length=40)
print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
trainer.validate(val_dataloaders=dm.val_dataloaders())

In [3]:
!pip freeze

absl-py @ file:///tmp/build/80754af9/absl-py_1607439979954/work
aiohttp @ file:///tmp/build/80754af9/aiohttp_1614360989480/work
async-generator==1.10
async-timeout==3.0.1
attrs @ file:///tmp/build/80754af9/attrs_1604765588209/work
backcall==0.2.0
bleach @ file:///tmp/build/80754af9/bleach_1611254400839/work
blinker==1.4
boto3==1.17.21
botocore==1.20.21
brotlipy==0.7.0
cachetools @ file:///home/conda/feedstock_root/build_artifacts/cachetools_1611555765219/work
certifi==2020.12.5
cffi @ file:///tmp/build/80754af9/cffi_1613246939562/work
chardet @ file:///tmp/build/80754af9/chardet_1605303159953/work
click==7.1.2
coverage @ file:///tmp/build/80754af9/coverage_1614614864940/work
cryptography @ file:///tmp/build/80754af9/cryptography_1613401440205/work
Cython @ file:///tmp/build/80754af9/cython_1614014840177/work
datasets==1.4.1
decorator==4.4.2
defusedxml==0.6.0
dill==0.3.3
entrypoints==0.3
filelock==3.0.12
fsspec @ file:///home/conda/feedstock_root/build_artifacts/fsspec_1614107550004/wor